In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb

# 1. Import data

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

# 2. Preprocessing

In [ ]:
df_all = pd.concat([df_train.drop(columns='Survived'),df_test],ignore_index=True)

In [ ]:
df_all

In [ ]:
df_all.info()

In [ ]:
# Search for missing data

msno.matrix(df=df_all, figsize=(20,14), color=(0,.3,.3))

In [ ]:
Embarked2 = []
for i in range(len(df_all)):
    if df_all["Embarked"][i] == "S":
        Embarked2.append(1)
    elif df_all["Embarked"][i] == "C":
        Embarked2.append(2)
    elif df_all["Embarked"][i] == "Q":
        Embarked2.append(3)
    else:
        Embarked2.append(4)
df_all = df_all.drop(columns="Embarked")
df_all["Embarked"] = Embarked2

In [ ]:
df_all["FamilySize"] = df_all["SibSp"] + df_all["Parch"] + 1

In [ ]:
# Add columns
ary = []
for i in range(len(df_all)):
    if df_all["FamilySize"][i] == 1:
        ary.append(1)
    else:
        ary.append(0)
df_all["isAlone"] = ary

In [ ]:
# Add columns
ary = []
for i in range(len(df_all)):
    if df_all["FamilySize"][i] == 2:
        ary.append(1)
    else:
        ary.append(0)
df_all["isPair"] = ary

In [ ]:
# Add columns
ary = []
for i in range(len(df_all)):
    if df_all["Embarked"][i] == 1:
        ary.append(1)
    else:
        ary.append(0)
df_all["English"] = ary

In [ ]:
# Add columns
ary = []
for i in range(len(df_all)):
    if df_all["Embarked"][i] == 2:
        ary.append(1)
    else:
        ary.append(0)
df_all["French"] = ary

In [ ]:
# Add columns
ary = []
for i in range(len(df_all)):
    if df_all["Embarked"][i] == 3:
        ary.append(1)
    else:
        ary.append(0)
df_all["Irish"] = ary

In [ ]:
# String label to categorical values

for i in range(df_all.shape[1]):
    if df_all.iloc[:,i].dtypes == object:
        lbl = LabelEncoder()
        lbl.fit(list(df_all.iloc[:,i].values))
        df_all.iloc[:,i] = lbl.transform(list(df_all.iloc[:,i].values))

In [ ]:
df_all

In [ ]:
df_all.info()

In [ ]:
# Missing data (Age, Fare) fill in median
for column in df_all.columns:
    df_all[column] = df_all[column].fillna(df_all[column].median())

In [ ]:
df_all

In [ ]:
df_all.info()

In [ ]:
df_train = pd.merge(df_all.iloc[df_train.index[0]:df_train.index[-1]+1],df_train['Survived'],left_index=True,right_index=True)
df_test = df_all.iloc[df_train.index[-1]+1:]

# 3. Check the correlation for each item

In [ ]:
df_train_corr = df_train.corr()
df_train_corr

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df_train_corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm');

# 4. Extract items with high correlation coefficient

In [ ]:
predictor_cols = []
for i in df_train_corr:
    if df_train_corr[i]['Survived'] > 0.01 or df_train_corr[i]['Survived'] < -0.01:
        innerName = df_train_corr[i].name
        if innerName != 'Survived':
            predictor_cols.append(innerName)
predictor_cols

# 5. Modeling

In [ ]:
x = DataFrame(df_train[predictor_cols])
t = DataFrame(df_train['Survived'])

x = np.array(x)
t = np.array(t)
t = t.ravel()

x = x.astype('float32')
t = t.astype('int32')

In [ ]:
# scaling
features = preprocessing.minmax_scale(x[:, :])

# split data for train and test
x_train, x_test, t_train, t_test = train_test_split(features, t.ravel(), test_size=0.2)

In [ ]:
#clf = neural_network.MLPClassifier(max_iter=200,
#                                   hidden_layer_sizes=(100,),
#                                   activation="relu",
#                                   solver="adam",
#                                   alpha=0.0001,
#                                   tol=0.0001,
#                                   verbose=True)
#clf.fit(x_train, t_train)

In [ ]:
# LightGBM
lgb_train = lgb.Dataset(x_train, t_train)
lgb_test = lgb.Dataset(x_test, t_test)
params = {
    'objective': 'binary',
    "metric":"binary_logloss",
    "learning_rate":0.01
}

gbm = lgb.train(
    params,
    train_set = lgb_train,
    valid_sets = [lgb_train, lgb_test],
    num_boost_round = 20000,
    early_stopping_rounds = 100,
    verbose_eval = 100
)

In [ ]:
#predict = clf.predict(x_test)
#accuracy = accuracy_score(t_test, predict)
#precision = precision_score(t_test, predict)
#recall = recall_score(t_test, predict)
#f1 = f1_score(t_test, predict)

In [ ]:
#accuracy

# 6. Prediction

In [ ]:
testData = DataFrame(df_test[predictor_cols])
testData = np.array(testData)
testData = testData.astype('float32')

# scaling
testData = preprocessing.minmax_scale(testData[:, :])

In [ ]:
#result = clf.predict(testData)

In [ ]:
prediction = gbm.predict(testData, num_iteration=gbm.best_iteration)

result = np.where(prediction < 0.51, 0, 1)

# 7. Make submission file

In [ ]:
outputArray = []
PassengerId = 100000
for i in range(len(result)):
    predict = result[i]
    innerArray = [PassengerId, predict]
    outputArray.append(innerArray)
    PassengerId += 1
df = pd.DataFrame(outputArray, columns=['PassengerId', 'Survived'])
df.to_csv(path_or_buf='submission.csv', index=False)

In [ ]:
df